In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import gensim
from gensim.models import KeyedVectors

Векторные представления русского и украинского языков:

In [3]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [4]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222575426101685),
 ('октябрь', 0.9095538854598999),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401416540145874)]

In [5]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999999403953552),
 ('липень', 0.9096440076828003),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992519378662109),
 ('жовтень', 0.8810408711433411),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431818008423)]

In [6]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.2757962942123413),
 ('ЖИЗНИВадим', 0.25203436613082886),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829231202602386),
 ('Ведущий-Алексей', 0.2443869560956955),
 ('Недопустимость', 0.24435284733772278),
 ('2Михаил', 0.23981399834156036),
 ('лексей', 0.23740756511688232),
 ('комплексн', 0.23695150017738342),
 ('персональ', 0.2368222028017044)]

In [7]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [8]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [9]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

Отображение пространства векторов:

In [10]:
from sklearn.linear_model import LinearRegression

In [11]:
mapping = LinearRegression().fit(X_train, Y_train)

In [12]:
print(mapping.score(X_train, Y_train))
print(mapping.coef_)

0.5440965035379247
[[-0.08416408  0.01545463  0.03119949 ...  0.04769695  0.09042673
   0.04564884]
 [-0.04712729  0.05719581  0.0421608  ...  0.01615214  0.04114803
   0.02863943]
 [ 0.14024085 -0.0116486   0.04446844 ... -0.01325072  0.05190985
  -0.05511026]
 ...
 [-0.01121384  0.0185028  -0.05139058 ... -0.04267926  0.06692278
  -0.00750024]
 [ 0.06275407  0.01538987  0.0291816  ...  0.04981291  0.06422392
  -0.06552111]
 [-0.0398456  -0.02388318 -0.03800555 ...  0.03092905  0.02589858
  -0.04698724]]


Соседи вектора слова "серпень" ("август") после линейного преобразования:

In [13]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531403541564941),
 ('июнь', 0.8402308225631714),
 ('март', 0.8385775089263916),
 ('сентябрь', 0.8331868648529053),
 ('февраль', 0.8311493992805481),
 ('октябрь', 0.8278172612190247),
 ('ноябрь', 0.8244150876998901),
 ('июль', 0.822899580001831),
 ('август', 0.8112363219261169),
 ('январь', 0.8022860288619995)]

In [14]:
ru_emb.most_similar(mapping.predict(uk_emb["кіт"].reshape(1, -1)))

[('кот', 0.7880541086196899),
 ('котик', 0.7193451523780823),
 ('кролик', 0.6709097027778625),
 ('щенок', 0.6688939332962036),
 ('заяц', 0.6628589630126953),
 ('песик', 0.6602395176887512),
 ('мышонок', 0.6596952676773071),
 ('котенок', 0.6593929529190063),
 ('котёнок', 0.6579437851905823),
 ('пес', 0.6566526293754578)]

Мы можем видеть, что окрестность этого векторного представления состоит из разных месяцев.

В качестве показателя качества мы будем использовать точность top-1, top-5 и top-10 (для каждого преобразованного украинского векторного представления мы подсчитываем, сколько правильных пар найдено в первых N ближайших соседях в русском пространстве векторных представлений).

In [15]:
"""
:args:
    pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
    mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
    topn = the number of nearest neighbours in destination embedding space to choose from
:returns:
    precision_val, float number, total number of words for those we can find right translation at top K.
"""

def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs): 
        mapped_vector = mapped_vectors[i]
        if ru in [x for (x,_) in ru_emb.most_similar([mapped_vector],topn=topn)]:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [16]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [17]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [18]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [19]:
print(precision_top1)
print(precision_top5)

0.6259541984732825
0.7913486005089059


In [20]:
""" 
:returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
"""

def learn_transform(X, Y):
    U, s, V = np.linalg.svd(np.matmul(X_train.T,Y_train))
    W = np.matmul(U,V)
    return W

In [21]:
W = learn_transform(X_train, Y_train)

print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6437659033078881
0.7989821882951654


In [22]:
"""
:args:
    sentence - sentence in Ukrainian (str)
:returns:
    translation - sentence in Russian (str)

* find ukrainian embedding for each word in sentence
* transform ukrainian embedding vector
* find nearest russian word and replace
"""

def translate(sentence):
    # YOUR CODE HERE
    words = sentence.split(" ")
    translation = []
    for word in words:
        try:
            emb = uk_emb[word]
            translation.append(ru_emb.most_similar([np.matmul(emb, W)], topn=1)[0][0])           
        except:
            translation.append(word)
    return " ".join(translation)

In [23]:
print(translate("яка гарна дівка"))

она красивая девка


In [24]:
print(translate("кіт зловив мишу"))

кот поймал мышку


In [25]:
print(translate("добре сало з горілкою , особливо з перцем"))

хорошо сало со водкой , особенно со перцем


In [27]:
with open("fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [28]:
uk_sentences

['\ufeffлисичка - сестричка і вовк - панібрат',
 'як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !',
 'дай мені огня " .',
 'а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .',
 'прибігла на поле , аж там пасуть хлопці бичків .',
 'вона і каже їм : " ей , хлопці !',
 'проміняйте мені бичка - третячка за маковий пирожок " .',
 'тії согласились ; так вона їм говорить : " смотріть же , ви не їжте зараз сього пирожка , а тоді уже розломите , як я заведу бичка за могилку ; а то ви його ні за що не розломите " .',
 'бачите вже - лисичка таки собі була розумна , що хоть кого да обманить .',
 'тії хлопці так і зробили , а лисичка як зайшла за могилу , да зараз у ліс і поверн

In [29]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: ﻿лисичка - сестричка і вовк - панібрат
dst: ﻿лисичка – сестричка и волк – панібрат

src: як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !
dst: как была себе лисичка , че и пошла раз к обеих бабы добывать огнь ; вошла во избу че и говорит : " хороший день тебе , бабушку !

src: дай мені огня " .
dst: дай мне огня " .

src: а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .
dst: а бабка только что вынула со печи самогончик со маком , сладкий , че и прогнула , чтобы он успокоился ; а лисичка ой и підгледала , че токмо что бабка свесилась во печь , чтобы прость огня , то лисичка сейчас ухватила самогончик че и дунула со хаты , че , пробежать , весь мак со его кринки 

src: де се ти набрала стільки риби ? "
dst: куда ой ты набрала столько рыбы ? "

src: вона каже : " наловила , вовчику - братику ! "
dst: она говорит : " наловила , плутишка – братику ! "

src: а собі на думці : " подожди , і я зроблю з тобою таку штуку , як і ти зо мною " .
dst: а себе по мнении : " вальс-бостон , и мной сделаю со тобой такую штуку , как и ты оло мной " .

src: - " як же ти ловила ? "
dst: – " как то ты ловила ? "

src: - " так , вовчику , уложила хвостик в ополонку , вожу тихенько да й кажу ; ловися , рибка , мала і велика !
dst: – " так , плутишка , уложила хвостик во прорубь , вожу тихонько че и говорю ; ловися , рыбка , имела и большая !

src: коли хочеш , то і ти піди , налови собі " .
dst: когда хочешь , то и ты пойди , налови себе " .

src: він побіг да зробив так , як казала лисичка .
dst: он побежал че сделал так , как говорила лисичка .

src: а лисичка стала за деревом да й дивиться ; коли у вовчика зовсім хвостик примерз , вона тоді побігла в село да й крич

Часть 2: Переводчик для английского и итальяского языков

Векторные представления английского и итальянских языков:

In [30]:
en_emb = KeyedVectors.load_word2vec_format("cc.en.300.vec", binary = False)
it_emb = KeyedVectors.load_word2vec_format("cc.it.300.vec", binary = False)

Перевод с английского на итальянский язык:

In [31]:
def load_word_pairs(filename):
    en_it_pairs = []
    en_vectors = []
    it_vectors = []
    
    with open(filename, "r") as inpf:
        for line in inpf:
            en, it = line.rstrip().split(" ")
            if en not in en_emb or it not in it_emb:
                continue
            en_it_pairs.append((en, it))
            en_vectors.append(en_emb[en])
            it_vectors.append(it_emb[it])
            
    return en_it_pairs, np.array(en_vectors), np.array(it_vectors)

In [32]:
en_it_train, X_train, Y_train = load_word_pairs("en-it.0-5000.txt")
en_it_test, X_test, Y_test = load_word_pairs("en-it.5000-6500.txt")

Получим две матрицы преобразования M1 и M2 для первых 300 пар и следующих 300 пар соответственно:

In [33]:
""" 
:returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
"""

def learn_transform(X, Y):
    U, s, V = np.linalg.svd(np.matmul(X_train.T,Y_train))
    W = np.matmul(U,V)
    return W

In [34]:
M1 = learn_transform(X_train[:300], Y_train[:300])
M2 = learn_transform(X_train[300:601], Y_train[300:601])

Вычислим точность для M1 и M2 на разных наборах пар:

In [35]:
"""
:args:
    pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
    mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
    topn = the number of nearest neighbours in destination embedding space to choose from
:returns:
    precision_val, float number, total number of words for those we can find right translation at top K.
"""
    
def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        mapped_vector = mapped_vectors[i]
        if ru in [x for (x, _) in it_emb.most_similar([mapped_vector], topn = topn)]:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [36]:
print("pair1 M1: {}".format(precision(en_it_train[:300], np.matmul(X_train[:300], M1))))
print("pair1 M2: {}".format(precision(en_it_train[:300], np.matmul(X_train[:300], M2))))
print("pair2 M1: {}".format(precision(en_it_train[300:601], np.matmul(X_train[300:601], M1))))
print("pair2 M2: {}".format(precision(en_it_train[300:601], np.matmul(X_train[300:601], M2))))

pair1 M1: 0.39666666666666667
pair1 M2: 0.39666666666666667
pair2 M1: 0.37209302325581395
pair2 M2: 0.37209302325581395


Для пары английский - итальянский языки получили точность 38%

Вычислим норму фробениуса для матрицы $|M1-M2|$:

In [37]:
M1_M2 = np.linalg.norm((M1-M2), ord ='fro')
M1_M2

0.0

Вычислим отношение фробениусовых норм $\frac{|M1-M2|}{|M1|}$:

In [38]:
M1_M2/np.linalg.norm(M1, ord ='fro')

0.0

Проверим ортогональность матриц:

In [39]:
print(np.matmul(M1, M1.T))

[[ 1.0000002e+00  8.3966096e-09  6.3929470e-08 ...  2.4390609e-08
  -1.0384546e-08  1.9656600e-08]
 [ 8.3966096e-09  9.9999976e-01  5.9413459e-08 ... -3.1495251e-08
  -3.3023371e-09  1.1393685e-08]
 [ 6.3929470e-08  5.9413459e-08  9.9999982e-01 ...  6.1542464e-09
   4.3215287e-08  1.4329093e-08]
 ...
 [ 2.4390609e-08 -3.1495251e-08  6.1542464e-09 ...  9.9999988e-01
  -9.1118091e-10  1.4785218e-08]
 [-1.0384546e-08 -3.3023371e-09  4.3215287e-08 ... -9.1118091e-10
   1.0000000e+00  6.1164398e-09]
 [ 1.9656600e-08  1.1393685e-08  1.4329093e-08 ...  1.4785218e-08
   6.1164398e-09  1.0000002e+00]]


M1 ортогональна.

In [40]:
print(np.matmul(M2, M2.T))

[[ 1.0000002e+00  8.3966096e-09  6.3929470e-08 ...  2.4390609e-08
  -1.0384546e-08  1.9656600e-08]
 [ 8.3966096e-09  9.9999976e-01  5.9413459e-08 ... -3.1495251e-08
  -3.3023371e-09  1.1393685e-08]
 [ 6.3929470e-08  5.9413459e-08  9.9999982e-01 ...  6.1542464e-09
   4.3215287e-08  1.4329093e-08]
 ...
 [ 2.4390609e-08 -3.1495251e-08  6.1542464e-09 ...  9.9999988e-01
  -9.1118091e-10  1.4785218e-08]
 [-1.0384546e-08 -3.3023371e-09  4.3215287e-08 ... -9.1118091e-10
   1.0000000e+00  6.1164398e-09]
 [ 1.9656600e-08  1.1393685e-08  1.4329093e-08 ...  1.4785218e-08
   6.1164398e-09  1.0000002e+00]]


M2 ортогональна.

Перевод с итальянского на английский язык:

In [41]:
def load_word_pairs(filename):
    it_en_pairs = []
    en_vectors = []
    it_vectors = []
    
    with open(filename, "r") as inpf:
        for line in inpf:
            it, en = line.rstrip().split(" ")
            if it not in it_emb or en not in en_emb:
                continue
            it_en_pairs.append((it, en))
            en_vectors.append(en_emb[en])
            it_vectors.append(it_emb[it])
            
    return it_en_pairs, np.array(it_vectors), np.array(en_vectors)

In [42]:
it_en_train, X_train, Y_train = load_word_pairs("it-en.0-5000.txt")
it_en_test, X_test, Y_test = load_word_pairs("it-en.5000-6500.txt")

Получим две матрицы преобразования M1 и M2 для первых 300 пар и следующих 300 пар соответственно:

In [43]:
M1 = learn_transform(X_train[:300], Y_train[:300])
M2 = learn_transform(X_train[300:601], Y_train[300:601])

Вычислим точность для M1 и M2 на разных наборах пар:

In [44]:
"""
:args:
    pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
    mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
    topn = the number of nearest neighbours in destination embedding space to choose from
:returns:
    precision_val, float number, total number of words for those we can find right translation at top K.
"""

def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        mapped_vector = mapped_vectors[i]
        if ru in [x for (x, _) in en_emb.most_similar([mapped_vector], topn=topn)]:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [45]:
print("pair1 M1: {}".format(precision(it_en_train[:300], np.matmul(X_train[:300], M1))))
print("pair1 M2: {}".format(precision(it_en_train[:300], np.matmul(X_train[:300], M2))))
print("pair2 M1: {}".format(precision(it_en_train[300:601], np.matmul(X_train[300:601], M1))))
print("pair2 M2: {}".format(precision(it_en_train[300:601], np.matmul(X_train[300:601], M2))))

pair1 M1: 0.5266666666666666
pair1 M2: 0.5266666666666666
pair2 M1: 0.5415282392026578
pair2 M2: 0.5415282392026578


Для пары итальянский - английский языки получили точность 53%

Вычислим норму фробениуса для матрицы  $|M1-M2|$:

In [46]:
M1_M2 = np.linalg.norm((M1-M2), ord = "fro")
M1_M2

0.0

Вычислим отношение фробениусовых норм |M1-M2|/|M1|

In [47]:
M1_M2/np.linalg.norm(M1, ord = "fro")

0.0

Проверим ортогональность матриц:

In [48]:
print(np.matmul(M1, M1.T))

[[ 9.9999970e-01 -1.2426998e-08  8.0515763e-09 ... -3.0165370e-09
   1.1689153e-08 -3.8375674e-08]
 [-1.2426998e-08  9.9999946e-01 -6.0931380e-09 ...  1.7028722e-08
   6.0050200e-09  2.6205896e-08]
 [ 8.0515763e-09 -6.0931380e-09  9.9999994e-01 ... -2.7677842e-09
  -2.3343157e-08 -4.8537574e-09]
 ...
 [-3.0165370e-09  1.7028722e-08 -2.7677842e-09 ...  1.0000005e+00
   2.6973359e-08 -3.4271423e-08]
 [ 1.1689153e-08  6.0050200e-09 -2.3343157e-08 ...  2.6973359e-08
   1.0000002e+00 -4.1501266e-08]
 [-3.8375674e-08  2.6205896e-08 -4.8537574e-09 ... -3.4271423e-08
  -4.1501266e-08  1.0000001e+00]]


M1 ортогональна.

In [49]:
print(np.matmul(M2, M2.T))

[[ 9.9999970e-01 -1.2426998e-08  8.0515763e-09 ... -3.0165370e-09
   1.1689153e-08 -3.8375674e-08]
 [-1.2426998e-08  9.9999946e-01 -6.0931380e-09 ...  1.7028722e-08
   6.0050200e-09  2.6205896e-08]
 [ 8.0515763e-09 -6.0931380e-09  9.9999994e-01 ... -2.7677842e-09
  -2.3343157e-08 -4.8537574e-09]
 ...
 [-3.0165370e-09  1.7028722e-08 -2.7677842e-09 ...  1.0000005e+00
   2.6973359e-08 -3.4271423e-08]
 [ 1.1689153e-08  6.0050200e-09 -2.3343157e-08 ...  2.6973359e-08
   1.0000002e+00 -4.1501266e-08]
 [-3.8375674e-08  2.6205896e-08 -4.8537574e-09 ... -3.4271423e-08
  -4.1501266e-08  1.0000001e+00]]


M2 ортогональна.